<H1> Лабораторная работа 5 </h1>

Загрузим необходимые библиотеки. Для использования метода дерева решений загрузим из библиотеки sklearn tree, для случайного леса - RandomForestClassifier

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Загрузим датасет и проведем валидацию данных

In [4]:
df = pd.read_csv("fraud.csv",sep=',')

features = ['trans_date_trans_time', 'amt', 'category', 'merchant', 'city', 'state', 'job', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'gender']
target = 'is_fraud'

fraud=df[df[target]==1]
not_fraud=df[df[target]==0]
not_fraud_down_sample=not_fraud.sample(n=15000)
under=pd.concat([fraud,not_fraud_down_sample])

x = under[features]
y = under[target]

x['hour']=pd.to_datetime(x['trans_date_trans_time'],dayfirst=True).dt.hour
x=x.drop(['trans_date_trans_time'],axis=1)
categorical_features = ['category', 'merchant', 'city', 'state', 'job', 'gender']
numeric_features = ['amt', 'lat', 'long', 'merch_lat', 'merch_long', 'city_pop', 'cc_num', 'hour']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
x = preprocessor.fit_transform(x)

<ipython-input-4-18e621d52c3a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['hour']=pd.to_datetime(x['trans_date_trans_time'],dayfirst=True).dt.hour


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.2)

<h3>Решающее дерево</h3>
Создадим модель и обучим ее

In [ ]:
model=tree.DecisionTreeClassifier()
model.fit(x_train,y_train)

DecisionTreeClassifier()

Сделаем предсказания и оценим их точность

In [ ]:
pred=model.predict(x_test)
print(metrics.classification_report(y_test,pred,digits=5))

              precision    recall  f1-score   support

         0.0    0.97956   0.98168   0.98062     12011
         1.0    0.87036   0.85723   0.86374      1723

    accuracy                        0.96607     13734
   macro avg    0.92496   0.91945   0.92218     13734
weighted avg    0.96586   0.96607   0.96596     13734



<h3> Метод случайного леса </h3>
Создадим модель найдем с помощьюе кросс-валидации наилучшие гиперпараметры(n_estimators - количество деревьев, max_features - количество признаков на дерево, max_depth - глубина обхода дерева, min_samples_split - минимальное количество транзакций для разделения узла, min_samples_leaf - минимальное количество транзакций для листьев, bootstrap - случайные выборки данных) и обучим с ними модель

In [8]:
model=RandomForestClassifier()
param_grid={
    'n_estimators': [100, 200, 300],
    'max_features': ['None', 'auto', 'sqrt', 'log2'],
    'max_depth': [4,6,8,10,30,40],
    'min_samples_split': [2,5,10],
    'min_samples_leaf': [1,2,4],
    'bootstrap': [True,False]
}
grid=GridSearchCV(model,param_grid,cv=3,verbose=2,n_jobs=1)
grid.fit(x_train,y_train)
best_params=grid.best_params_
best_model=RandomForestClassifier(**best_params)
best_model.fit(x_train,y_train)
pred=best_model.predict(x_test)
print(metrics.classification_report(y_test,pred,digits=5))

Fitting 3 folds for each of 1296 candidates, totalling 3888 fits
[CV] END bootstrap=True, max_depth=4, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=4, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=4, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=4, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=4, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=4, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=4, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total tim

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=4, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=4, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s


...

[CV] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   2.4s


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=False, max_depth=40, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   2.5s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.7s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.8s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.7s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.6s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.6s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.5s
[CV] END bootstrap=False, max_depth=40, 

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
972 fits failed out of a total of 3888.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
972 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/u

              precision    recall  f1-score   support

           0    0.95231   0.99674   0.97402     11980
           1    0.96686   0.65553   0.78133      1736

    accuracy                        0.95356     13716
   macro avg    0.95959   0.82614   0.87767     13716
weighted avg    0.95415   0.95356   0.94963     13716

